In [1]:
import base64
import cv2
import pika

In [2]:
#pika is a RabbitMQ python library 
#Producer and consumer.
#Between them there is the queue. The producer sends a message to the queue but not directly, instead,
#it uses an exchange. Publishing a message means that the publisher sends a message to an exchange and it forwards the message
#to the queue. Consuming a message means that the consumer picks the message from the queue and consumes it.
#The exchange is important because if we have more queue, through the 'binding_key', it knows who to send it to.
#An exchange connects to a queue through a binding key. 
#To send a message, the producer needs to specify a 'routing key'.
#DIFFERENT EXCHANGES:
#Fanout sends the messages to all the queues it knows about.
#Direct sends the messages where 'routing key'=='binding key'
#Topic sends the messages where there is a partial match. for example 'red.*' will send it either at 'red.blue' and 'red.green'
#Header uses messages header instead of 'routing key'
#Default(nameless) send the message where 'routing key' == 'queue name'

#SUM-UP
#Producer emits messages to exchange
#consumer receives messages from queue
#Binding connects an exchange with a queue using binding key
#exchange compares routing key with binding key.
#messages distribution depends on exchange type

Import Libraries

Set Connection

In [3]:
host = '192.168.56.102' #La mia macchina virtuale
#host = '192.168.1.172'
exchange_name = 'camera'
routing_key_name = 'view'
routing_key_security_name= 'security'
#The BlockingConnection creates a layer on top of Pika’s asynchronous core providing methods
#that will block until their expected response has returned.
connection = pika.BlockingConnection(pika.ConnectionParameters(host))
channel = connection.channel()

In [4]:
channel.exchange_declare(exchange=exchange_name, exchange_type='topic')

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>

Starting Camera

In [5]:
cam = cv2.VideoCapture(0)
frame_number = -1

cv2.namedWindow("Camera View")
print('Sending messages')

while True:
    #Show picture
    ret, frame = cam.read()
    cv2.imshow("Camera View", frame)
    
    key = cv2.waitKey(50) & 0xFF
    
    if key % 256 == 27:
        #ESC pressed
        print("Closing...")
        break
    if key % 256 == 32:
        #SPACE pressed
        encodedImg = base64.b64encode(frame)
        
        channel.basic_publish(exchange=exchange_name,
                              routing_key=routing_key_security_name,
                              body=encodedImg)
        
    #Don't send all the frames
    frame_number += 1
    if frame_number % 5 == 0:
        frame_number = 0
        
        #Pre_elaboration
        encodedImg = base64.b64encode(frame)

        #Send to MQTT broker
        channel.basic_publish(exchange=exchange_name,
                              routing_key=routing_key_name,
                              body=encodedImg)

Sending messages
Closing...


Close Stuff

In [6]:
connection.close()      
cam.release()
cv2.destroyAllWindows()